<a href="https://colab.research.google.com/github/takyaC/Finance/blob/main/EstimateAndActual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yahoo_fin

In [ ]:
pip install pandas

In [ ]:
pip install requests_html

In [47]:
from yahoo_fin.stock_info import get_analysts_info, get_cash_flow, get_income_statement, get_earnings_history, get_earnings, get_next_earnings_date
import pandas

In [152]:
tickers = [
           "SQ","WKHS"
#  "AAPL","AFRM","AMZN","ANTM","APPS","BMY","BNTX","CCL","CRWD","DAL",
#  "DKNG","DOCU","EBS","FUTU","FVRR","GE","GOOGL","IAC","JPM","MRNA",
#  "MSFT","NIO","ORCL","PINS","PLUG","PTON","RBLX","RKT","ROKU","ROOT",
#  "RPRX","SBLK","SQ","TSLA","U","WKHS","WORK","ZI","ZM"
 ]

In [159]:
data_of_ticker = []

for ticker in tickers:
    try:
      # アナリスト予想（予想EPS、予想売上）
      ai = get_analysts_info(ticker)
      earnings_estimate = ai["Earnings Estimate"].iloc[[1], [1, 3, 4]]
      revenue_estimate = ai["Revenue Estimate"].iloc[[1], [1, 3, 4]]

      # EPS履歴
      earings = get_earnings(ticker)
      quarterly_results_row = len(earings["quarterly_results"]) -1
      earings_last_results = earings["quarterly_results"].iloc[quarterly_results_row, 2] # 右下のデータ(最終？Q、実績)を取得      

      # 損益計算書（四半期売上）
      ist = get_income_statement(ticker, False)
      total_revenue = ist.loc[["totalRevenue"], ist.columns[0]]

      # 単位付与(100万/１０億)
      total_revenue_by_million = total_revenue.values / 1000 / 1000
      if total_revenue_by_million < 1000:
        converted_total_revenue = str(round(total_revenue_by_million[0], 2)) + "M"
      else:
        converted_total_revenue = str(round((total_revenue_by_million[0] / 1000), 2)) + "B"
        # a = total_revenue_by_million[0]
        # b = a / 1000
        # c = round(b, 2)
        # print(a , type(a))
        # print(b , type(b))
        # print(c , type(c))
        # print(round((total_revenue_by_million[0] / 1000), 2))
      
      # 損益計算書（年間売上）
      isty = get_income_statement(ticker)
      total_revenue_yearly = isty.loc[["totalRevenue"], isty.columns[0]]

      # 単位付与(100万/１０億)
      total_revenue_yearly_by_million = total_revenue_yearly.values / 1000 / 1000
      if total_revenue_yearly_by_million < 1000:
        converted_total_revenue_yearly_by_million = str(round(total_revenue_yearly_by_million[0], 2)) + "M"
      else:
        converted_total_revenue_yearly_by_million = str(round((total_revenue_yearly_by_million[0] / 1000), 2)) + "B"

      # キャッシュフロー計算書（純利益、営業キャッシュフロー、リスク（純利益<営業キャッシュフローが正常)）
      cfy = get_cash_flow(ticker)
      net_income_yearly = cfy.loc[["netIncome"], cfy.columns[0]]
      operating_cash_flow_yearly = cfy.loc[["totalCashFromOperatingActivities"], cfy.columns[0]]
      is_risk_cleard  = "OK" if operating_cash_flow_yearly.values > net_income_yearly.values else "NG"

      # 営業キャッシュフローマージン (15%以上推奨)
      cash_flow_margin_yearly_before = (operating_cash_flow_yearly.values / total_revenue_yearly.values)
      cash_flow_margin_yearly = round(cash_flow_margin_yearly_before[0], 2)


      # 次回決算発表日
      next_earnings_date = get_next_earnings_date(ticker)

      # 設定
      data = []
      data.extend([ticker])

      data.extend([earings_last_results]) # 前四半期 EPS
      data.extend([converted_total_revenue]) # 前四半期 売上
      data.extend([earnings_estimate.values[0][0]]) # 今四半期予想 EPS
      data.extend([revenue_estimate.values[0][0]]) # 今四半期予想 売上
      data.extend([earnings_estimate.values[0][1]]) # 今年度予想 EPS
      data.extend([revenue_estimate.values[0][1]]) # 今年度予想 売上
      data.extend([earnings_estimate.values[0][2]]) # 次年度予想 EPS
      data.extend([revenue_estimate.values[0][2]]) # 次年度予想 売上
      # data.extend([converted_total_revenue_yearly_by_million]) # 前年度売上

      data.extend([is_risk_cleard])
      data.extend([cash_flow_margin_yearly])
      
      data.extend(["-"]) # 空列
      data.extend([next_earnings_date]) # 次回決算発表日
      data_of_ticker.append(data)

      print(data)
    except Exception as e:
      print(e)
      data_of_ticker.append([ticker])
      

['SQ', 0.24, '3.16B', 0.16, '3.28B', 1.23, '13.76B', 1.88, '16.35B', 'OK', 0.04, '-', datetime.datetime(2021, 5, 6, 0, 0)]
['WKHS', -0.13, '0.65M', -0.17, '2.71M', -0.58, '112.44M', -0.13, '295.57M', 'NG', -50.47, '-', datetime.datetime(2021, 3, 1, 0, 0)]


In [160]:
pandas.DataFrame(data_of_ticker, columns=[
        "銘柄",
        "前四半期EPS",
        "前四半期売上",
        "今四半期予想　EPS", 
        "今四半期予想　売上",
        "今年度予想　EPS", 
        "今年度予想　売上",
        "次年度予想　EPS",
        "次年度予想　売上",
        "リスク",
        "営業キャッシュフローマージン",
        "-",
        "次回決算発表日"])


,銘柄,前四半期EPS,前四半期売上,今四半期予想 EPS,今四半期予想 売上,今年度予想 EPS,今年度予想 売上,次年度予想 EPS,次年度予想 売上,リスク,営業キャッシュフローマージン,-,次回決算発表日
0,SQ,0.24,3.16B,0.16,3.28B,1.23,13.76B,1.88,16.35B,OK,0.04,-,2021-05-06
1,WKHS,-0.13,0.65M,-0.17,2.71M,-0.58,112.44M,-0.13,295.57M,NG,-50.47,-,2021-03-01
